In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터셋을 로드한다.
df = pd.read_csv("../merged_dataset.csv")

# 우선 훈련데이터, 검증데이터, 테스트데이터로 나눠야 함
# 먼저 train / temp 로 나눔 (stratify로 비율 유지)
train_df, temp_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["label"],
    random_state=42
)

# temp 데이터를 다시 validation / test 로 나눔
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=42
)

# CSV 파일로 저장
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

## 예측 모델 학습

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import os
os.environ["WANDB_DISABLED"] = "true"

# 1. 모델과 토크나이저 로드
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 2. pandas로 데이터 로드
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")

# 3. HuggingFace Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 4. 토크나이징 함수 정의
def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# 5. 토크나이징 적용
train_tokenized = train_dataset.map(tokenize_fn)
val_tokenized = val_dataset.map(tokenize_fn)

# 6. 학습 설정
training_args = TrainingArguments(
    output_dir="./bert-smishing-model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs"
)


# 7. Trainer 구성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

trainer.train()

# 8. 모델 저장
trainer.save_model("./bert-smishing-model")
tokenizer.save_pretrained("./bert-smishing-model")


## test.csv 로 테스트 실행

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# test.csv 불러오기
test_df = pd.read_csv("test.csv")
test_dataset = Dataset.from_pandas(test_df)

# 토크나이징
def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_test = test_dataset.map(tokenize_fn)

# 예측
predictions = trainer.predict(tokenized_test)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# 결과 출력
print(classification_report(y_true, y_pred, target_names=["정상", "스미싱"]))


## 단일 추론 테스트

In [ ]:
from transformers import pipeline

# 파이프라인 로딩
classifier = pipeline("text-classification", model="./bert-smishing-model", tokenizer="./bert-smishing-model")

# 예제 추론
result = classifier("[Web발신] 국세청 환급금 조회 링크를 클릭하세요 http://scam.link")
print(result)

In [ ]:
# 예제 추론
result = classifier("[SK텔레콤] 이심(eSIM) 무료 간편 교체 안내드립니다.[Web발신][SKT] 이심(eSIM) 무료 간편 교체 안내 유심 교체를 예약해 주신 고객님께 안내드립니다. 고객님은 현재 이심(eSIM)으로 교체 가능한 기기를 사용하고 계십니다. 이심 또는 유심 교체 중 고객님께서 원하시는 방식을 선택하실 수 있습니다. 이심은 휴대폰에 내장된 디지털 SIM을 이용하는 방식으로 유심과 동일한 역할을 합니다. 이심 교체를 신청하시면 유심을 기다리거나 매장에 방문하실 필요 없이 바로 교체하실 수 있습니다. ■ 이심 간편 교체 안내 - 1회 교체 비용은 무료입니다. - 이심 교체가 완료되면 유심 교체 예약은 자동으로 취소됩니다. * 이심 교체 가능 시간: 월~토요일 오전 8시~오후 9시 (공휴일 제외) ■ 이심 교체 전 확인 사항 - Wi-Fi가 연결된 상태에서 신청 가능합니다. - 연락처 백업, 티머니 잔액 확인, 다른 SIM 사용 제한 해지(기존에 설정한 고객님) 등은 아래 URL의 유의 사항을 확인해 주시기 바랍니다. ▶ 이심 간편 교체 신청 바로 가기: https://m.shop.tworld.co.kr/nf/index_nf_yp_change.html 자세한 설명이 필요하시면 T 월드 웹/앱 또는 고객센터에 문의해 주시기 바랍니다. ■ 문의: 고객센터 1525(무료) 이용에 불편함이 없도록 최선을 다하겠습니다. SK텔레콤 드림")
print(result)